## Find Empirical Maximin Share Guarantee

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Assumptions
- n agents
- 8 players per team
- total pool of $8n$ players

### Cardinal Constraints
- 1 QB (quarterback)
- 2 RB (running back)
- 2 WR (wide receiver)
- 1 TE (tight end)
- 1 K (kicker)
- 1 DST (defense)

In [18]:
# number of agents
n = 2

# number of players per team
m = 8

# cardinal constraints
constraints = {"QB": 1,
              "RB": 2,
              "WR": 2,
              "TE": 1,
              "K": 1,
              "DST": 1}

# import df
df = pd.read_csv("FantasyPros_Fantasy_Football_Points.csv")

In [9]:
class Player():
    """ Class to represent American football players """
    
    def __init__(self, idx, name, position, valuation=0):
        self.id = idx
        self.name = name
        self.position = position
        self.selected = False
        self.valuation = valuation
        
    def isAvailable(self):
        return not self.selected
    
    def select(self):
        assert not self.selected
        self.selected = True
        
    def getPosition(self):
        return self.position
    
    def getValuation(self):
        return self.valuation
    
    def __str__(self):
        return f"{self.name} ({self.position})"
    
    def __repr__(self):
        return f"{self.name} ({self.position})"
    
    def __lt__(self, other):
        return self.valuation < other.valuation
    
    def __gt__(self, other):
        return self.valuation > other.valuation

In [16]:
def create_pool(df, constraints, n, m, random=False):
    """ Create pool of players to allocate among agents"""
    
    # instantiate pool
    # with exactly the number of total players that will be selected
    df_pool = pd.DataFrame(columns=df.columns)
    
    for position, quota in constraints.items():
        
        if random:
            df_pool = pd.concat([df_pool,
                        df.iloc[np.random.choice(df[df["Pos"] == position].index, size=quota*2, replace=False)]],
                       ignore_index=True)
            
        else:
            df_pool = pd.concat([df_pool,
                            df[df["Pos"] == position].sort_values("TTL").head(quota*n)],
                          ignore_index=True)
            
    # make valuations additive (adding up to 100)
    df_pool["VAL"] = (df_pool["TTL"] / df_pool["TTL"].sum()) * 100

    pool = []
    for i in range(n):
        for j in range(m):
            idx = i*m+j
            player = Player(idx, df_pool.iloc[idx]["Player"], df_pool.iloc[idx]["Pos"], df_pool.iloc[idx]["VAL"])
            pool.append(player)
            
    return pool

In [13]:
# algorithm u 
# https://gist.github.com/olooney/8607faf1ee609b7c4da26f41f766a977

def form_bundles(ns, m):
    """Calculate all m-partitions that can be made of ns"""
    def visit(n, a):
        ps = [[] for i in range(m)]
        for j in range(n):
            ps[a[j + 1]].append(ns[j])
        return ps

    def f(mu, nu, sigma, n, a):
        if mu == 2:
            yield visit(n, a)
        else:
            for v in f(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v
        if nu == mu + 1:
            a[mu] = mu - 1
            yield visit(n, a)
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                yield visit(n, a)
        elif nu > mu + 1:
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = mu - 1
            else:
                a[mu] = mu - 1
            if (a[nu] + sigma) % 2 == 1:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v

    def b(mu, nu, sigma, n, a):
        if nu == mu + 1:
            while a[nu] < mu - 1:
                yield visit(n, a)
                a[nu] = a[nu] + 1
            yield visit(n, a)
            a[mu] = 0
        elif nu > mu + 1:
            if (a[nu] + sigma) % 2 == 1:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] < mu - 1:
                a[nu] = a[nu] + 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = 0
            else:
                a[mu] = 0
        if mu == 2:
            yield visit(n, a)
        else:
            for v in b(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v

    n = len(ns)
    a = [0] * (n + 1)
    for j in range(1, m + 1):
        a[n - m + j] = j - 1
    return f(m, n, 0, n, a)

In [20]:

# identical for all players since all players change the same valuation
def maximin(bundles, quotas):
    """Find maximin share valuation and allocation"""
    
    # keep track of maximin valuation
    maximin_valuation = 0
    maximin_bundle = []
    
    counter = 1
    for bundle_group in bundles:
        # store bundle less valued of the two
        bundle_valuations = []
        
        for bundle in sorted(bundle_group, key = lambda x: len(x)):
            if len(bundle) < 2: break
                
            # sort bundle by player decreasing valuation
            bundle = sorted(bundle, key=lambda x: -x.valuation)
            
            # keep track of filled positions
            filled = {"QB": [],
                     "RB": [],
                     "WR": [],
                     "TE": [],
                     "K": [],
                     "DST": []}
            
            for player in bundle:
                # if player position's quota hasn't been filled, add player to this agent's team
                # therefore, agent's total valuation will include their valuation for this player
                if len(filled[player.position]) < quotas[player.position]:
                    filled[player.position].append(player.valuation)
                    
            # calculate valuation for current bundle
            # adding all valuations from players in agent's team given quota constraints
            bundle_valuation = 0
            for key, val in filled.items():
                bundle_valuation += np.sum(val)
                
            bundle_valuations.append(bundle_valuation)
                
        # for each bundle group, store the min valuation of the (min_valuation)
        # valuations.append(min_valuation)
        if len(bundle_valuations) > 1:
            min_valuation = np.min(np.array(bundle_valuations))
            if min_valuation > maximin_valuation:
                maximin_valuation = min_valuation
                maximin_bundle = bundle_group
        counter += 1
        
    return maximin_valuation, maximin_bundle

In [22]:
# find all possible bundles among n agents
# using pool of best players
pool = create_pool(df, constraints, n, m)
bundles = form_bundles(pool, n)

# find MMS
valuations, mbundles = maximin(bundles, constraints)

In [ ]:
# run many randon simulations of possible MMS
# using pool of random players

simulations = 10000
mms = []

for i in range(simulations):
    pool = create_pool(df, constraints, n, m)
    bundles = form_bundles(pool, n)
    valuation, _ = maximin(bundles, constraints)
    mms.append(valuation)